In [136]:
import pandas as pd
import numpy as np
import markovify
import ast
import random
import operator
import bisect
import json
import copy

In [8]:
df = pd.read_csv("melody_chords.csv")
df.head()

,Unnamed: 0,track_id,serial,pitch,beat,bar,chord
0,0,1,0,65.0,1,0,Bb6
1,1,1,1,63.0,2,0,Bb6
2,2,1,2,58.0,2,0,Bb6
3,3,1,3,61.0,3,0,Bb6
4,4,1,4,63.0,4,0,Bb6


In [13]:
all_chords = set(df["chord"].unique())

In [22]:
notes_for_chord = {}

for chord in all_chords:
    chord_df = df[df["chord"] == chord]
    notes = set(chord_df["pitch"].unique())

    notes_for_chord[chord] = notes


In [48]:
data = []

for track_id in df["track_id"].unique():
    track_df = df[df["track_id"] == track_id]

    notes = track_df["pitch"].tolist()

    data.append(notes)

In [252]:
BEGIN = "___BEGIN__"
END = "___END__"

class MelodyChain(markovify.Chain):
    def gen(self, chords, init_state=None):
        state = init_state or (markovify.chain.BEGIN,) * self.state_size
        n = len(chords)
        i = 0

        while i < n:
            next_word = self.move(state, chords[i])
            yield next_word
            state = tuple(state[1:]) + (next_word,)
            i += 1

    def move(self, state, chord):
        """
        Given a state, choose the next item at random.
        """
        if self.compiled:
            choices, cumdist = self.model[state]
        elif state == tuple([ markovify.chain.BEGIN ] * self.state_size):
            choices = self.begin_choices
            cumdist = self.begin_cumdist
        else:
            choices, weights = zip(*self.model[state].items())
            cumdist = list(markovify.chain.accumulate(weights))
        
        choices = list(choices)
        
        rem = []
        if chord in all_chords:
            i = 0
            for c in choices:
                if not c in notes_for_chord[chord]:
                    rem.append(i)
                i += 1

        else:
            if END in choices:
                rem.append(choices.index(END))

        for i in sorted(rem, reverse=True):
            if len(cumdist) > 1:
                del choices[i]
                if i < len(cumdist):
                    prev = 0
                    if i > 0:
                        prev = cumdist[i - 1]

                    val = cumdist[i] - prev
                    
                    for j in range(i + 1, len(cumdist)):
                        cumdist[j] -= val
                    
                    del cumdist[i]            

        r = random.random() * cumdist[-1]
        selection = choices[bisect.bisect(cumdist, r)]
        return selection

In [253]:
model = MelodyChain(data, state_size=2)

In [254]:
chords = pd.read_csv("chords.csv", names=["tune", "chords"], skiprows=1)

chords["notes"] = ""
chords.head()


,tune,chords,notes
0,0,"[('2', 'Gm7'), ('2', 'C7b9'), ('2', 'Am7'), ('...",
1,1,"[('2', 'Gm7'), ('2', 'C7b9'), ('2', 'Am7'), ('...",
2,2,"[('2', 'Cm7'), ('2', 'F7'), ('1', 'Bbmaj7'), (...",
3,3,"[('2', 'Gm7'), ('2', 'C7b9'), ('2', 'Am7'), ('...",
4,4,"[('1', 'Cm6'), ('1', 'Cm6'), ('1', 'Cm7#9'), (...",


In [255]:
for (i, row) in chords.iterrows():
    prog = []
    for c in eval(row["chords"]):
        prog.append(c[1])

    notes = [x for x in model.gen(prog)]
    row["notes"] = notes
    chords.loc[i] = row

IndexError: list index out of range

In [198]:
chords.head()

,tune,chords,notes
0,0,"[('2', 'Gm7'), ('2', 'C7b9'), ('2', 'Am7'), ('...","[58.0, 55.0, 57.0, 58.0, 53.0, 58.0, 57.0, 69...."
1,1,"[('2', 'Gm7'), ('2', 'C7b9'), ('2', 'Am7'), ('...",
2,2,"[('2', 'Cm7'), ('2', 'F7'), ('1', 'Bbmaj7'), (...",
3,3,"[('2', 'Gm7'), ('2', 'C7b9'), ('2', 'Am7'), ('...",
4,4,"[('1', 'Cm6'), ('1', 'Cm6'), ('1', 'Cm7#9'), (...",
